# Customer Segmentation - FIXED VERSION - Part 2

## Feature Engineering & RFM Analysis

This notebook continues from Part 1 with the cleaned data.

<h2 style="color:darkmagenta;text-align: center; background-color: AliceBlue;padding: 20px;">5. Feature Engineering - RFM Analysis</h2><a id="5"></a>

### 🔴 CRITICAL FIX: Correct RFM Calculation

**The Original Error:**
```python
# WRONG - This calculates time span, NOT recency!
Recency = TransactionDate2 - TransactionDate1  # First to last transaction
```

**The Correct Approach:**
```python
# CORRECT - Days since last transaction
Recency = (analysis_date - last_transaction_date).days
```

---

### What is RFM Analysis?

<img src="https://miro.medium.com/max/638/0*JddVeZpHXdElEec_" alt="RFM" width="600">

**RFM** is a proven marketing analysis technique:

1. **Recency (R)** - How recently did the customer transact?
   - Lower is better (recent = more engaged)
   - Measured in days since last transaction
   - Example: 5 days ago = very engaged, 365 days ago = at risk

2. **Frequency (F)** - How often do they transact?
   - Higher is better (frequent = loyal)
   - Measured as count of transactions
   - Example: 50 transactions = loyal, 1 transaction = new/inactive

3. **Monetary (M)** - How much do they spend?
   - Higher is better (big spender = valuable)
   - Measured as total or average transaction amount
   - Example: ₹100,000 total = high value, ₹1,000 total = low value

### Why RFM Works:
- **Simple yet powerful** - Easy to understand and implement
- **Actionable** - Directly informs marketing strategy
- **Proven** - Used by major retailers and banks worldwide
- **Predictive** - Strong indicator of future behavior

### Additional Features:
Beyond basic RFM, we'll include:
- Account Balance (financial status)
- Customer Age (demographic)
- Gender (demographic)
- Location (geographic)

In [ ]:
# Define analysis date - the reference point for calculating recency
# Using the last date in our dataset as the "present"

ANALYSIS_DATE = df['TransactionDate'].max()

print("📅 Analysis Reference Date:")
print("=" * 80)
print(f"Analysis Date: {ANALYSIS_DATE.date()}")
print(f"\nThis is the reference point for calculating 'Recency'")
print(f"Example: If a customer's last transaction was on {(ANALYSIS_DATE - pd.Timedelta(days=10)).date()}")
print(f"         Their recency = 10 days")
print("\nWhy use the last date in dataset?")
print("  - Represents the 'present' in our historical data")
print("  - All customers measured from same reference point")
print("  - In production, would use TODAY's date for new predictions")

### Aggregation Strategy

We need to transform **transaction-level** data to **customer-level** data.

**Why?** Because we want to segment CUSTOMERS, not transactions.

**How we aggregate each feature:**

| Feature | Aggregation | Reason |
|---------|-------------|--------|
| TransactionDate | max() | Last transaction date (for recency) |
| TransactionID | count() | Number of transactions (frequency) |
| TransactionAmount | sum() | Total spending (monetary) |
| TransactionAmount | mean() | Average transaction size |
| CustAccountBalance | last() | Current balance |
| CustomerAge | first() | Age (static demographic) |
| CustGender | first() | Gender (static demographic) |
| CustLocation | first() | Location (static demographic) |

In [ ]:
print("🔧 Creating Customer-Level Dataset")
print("=" * 80)
print(f"Starting with: {len(df):,} transactions")
print(f"From: {df['CustomerID'].nunique():,} unique customers")
print("\nAggregating to one row per customer...\n")

# Create customer-level aggregated dataset
customer_df = df.groupby('CustomerID').agg({
    # For Recency calculation
    'TransactionDate': 'max',  # Last transaction date
    
    # Frequency
    'TransactionID': 'count',  # Number of transactions
    
    # Monetary
    'TransactionAmount (INR)': ['sum', 'mean', 'std'],  # Total, average, volatility
    
    # Account information
    'CustAccountBalance': ['last', 'mean'],  # Current and average balance
    
    # Demographics (static - same for all transactions)
    'CustomerAge': 'median',  # Median age across transactions
    'CustGender': 'first',
    'CustLocation': 'first'
}).reset_index()

# Flatten multi-level column names
customer_df.columns = ['_'.join(col).strip('_') if col[1] else col[0] 
                       for col in customer_df.columns.values]

print(f"✓ Created customer dataset: {len(customer_df):,} customers")
print("\nColumn names after aggregation:")
for col in customer_df.columns:
    print(f"  - {col}")

In [ ]:
# Rename columns for clarity
customer_df = customer_df.rename(columns={
    'TransactionDate_max': 'LastTransactionDate',
    'TransactionID_count': 'Frequency',
    'TransactionAmount (INR)_sum': 'MonetaryTotal',
    'TransactionAmount (INR)_mean': 'MonetaryAvg',
    'TransactionAmount (INR)_std': 'MonetaryStd',
    'CustAccountBalance_last': 'AccountBalance',
    'CustAccountBalance_mean': 'AccountBalanceAvg',
    'CustomerAge_median': 'Age',
    'CustGender_first': 'Gender',
    'CustLocation_first': 'Location'
})

print("✓ Columns renamed for clarity")
customer_df.head()

### 🎯 Calculate Recency - THE CORRECT WAY

**Formula:**
```
Recency = (Analysis_Date - Last_Transaction_Date).days
```

**Interpretation:**
- Recency = 1 → Transaction yesterday (VERY engaged)
- Recency = 30 → Transaction a month ago (Moderately engaged)
- Recency = 365 → Transaction a year ago (At risk/dormant)

**Why this matters for segmentation:**
- Recent customers are more likely to respond to marketing
- High recency indicates churn risk
- Recency is often the strongest predictor of future behavior

In [ ]:
# Calculate Recency - CORRECT METHOD
print("📊 Calculating Recency (Days Since Last Transaction)")
print("=" * 80)

customer_df['Recency'] = (ANALYSIS_DATE - customer_df['LastTransactionDate']).dt.days

print(f"\n✓ Recency calculated correctly!")
print(f"\nRecency Statistics:")
print(f"  Minimum: {customer_df['Recency'].min()} days (most recent)")
print(f"  Maximum: {customer_df['Recency'].max()} days (longest time since transaction)")
print(f"  Mean: {customer_df['Recency'].mean():.1f} days")
print(f"  Median: {customer_df['Recency'].median():.1f} days")

# Interpret the distribution
recent = (customer_df['Recency'] <= 30).sum()
moderate = ((customer_df['Recency'] > 30) & (customer_df['Recency'] <= 90)).sum()
at_risk = (customer_df['Recency'] > 90).sum()

print(f"\nRecency Distribution:")
print(f"  Recent (≤30 days): {recent:,} customers ({recent/len(customer_df)*100:.1f}%)")
print(f"  Moderate (31-90 days): {moderate:,} customers ({moderate/len(customer_df)*100:.1f}%)")
print(f"  At Risk (>90 days): {at_risk:,} customers ({at_risk/len(customer_df)*100:.1f}%)")

In [ ]:
# Visualize Recency distribution
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.hist(customer_df['Recency'], bins=50, color='purple', edgecolor='black')
plt.xlabel('Recency (days)')
plt.ylabel('Number of Customers')
plt.title('Recency Distribution')
plt.axvline(customer_df['Recency'].median(), color='red', linestyle='--', 
            label=f'Median: {customer_df["Recency"].median():.0f} days')
plt.legend()

plt.subplot(1, 3, 2)
plt.hist(customer_df['Frequency'], bins=50, color='green', edgecolor='black')
plt.xlabel('Frequency (# transactions)')
plt.ylabel('Number of Customers')
plt.title('Frequency Distribution')
plt.axvline(customer_df['Frequency'].median(), color='red', linestyle='--',
            label=f'Median: {customer_df["Frequency"].median():.0f} transactions')
plt.legend()

plt.subplot(1, 3, 3)
plt.hist(customer_df['MonetaryTotal'], bins=50, color='orange', edgecolor='black')
plt.xlabel('Monetary Total (INR)')
plt.ylabel('Number of Customers')
plt.title('Monetary Distribution')
plt.axvline(customer_df['MonetaryTotal'].median(), color='red', linestyle='--',
            label=f'Median: ₹{customer_df["MonetaryTotal"].median():,.0f}')
plt.legend()

plt.tight_layout()
plt.show()

print("\n📈 Distribution Analysis:")
print("  - All three metrics show right-skewed distributions")
print("  - Most customers have low frequency and monetary values")
print("  - Few high-value customers (long tail)")
print("  - This is typical for banking/retail customer data")

In [ ]:
# Handle MonetaryStd NaN values
# When a customer has only 1 transaction, std is NaN
print("🔧 Handling MonetaryStd NaN values...")
nan_count = customer_df['MonetaryStd'].isnull().sum()
print(f"  Customers with NaN MonetaryStd: {nan_count:,}")
print(f"  (These are customers with only 1 transaction)")

# Fill NaN with 0 (no variation because only 1 transaction)
customer_df['MonetaryStd'] = customer_df['MonetaryStd'].fillna(0)
print("  ✓ Filled NaN with 0 (no variation for single transactions)")

### 📊 RFM Summary Statistics

Let's understand the typical customer in our dataset:

In [ ]:
# Create comprehensive RFM summary
rfm_features = ['Recency', 'Frequency', 'MonetaryTotal', 'MonetaryAvg', 
                'AccountBalance', 'Age']

print("📊 Customer Metrics Summary")
print("=" * 80)
summary = customer_df[rfm_features].describe().round(2)
summary

In [ ]:
# Analyze correlations between RFM features
print("\n🔗 Feature Correlations")
print("=" * 80)
print("Understanding how features relate to each other:")
print("  - Strong correlation: Features move together (may be redundant)")
print("  - Weak correlation: Features capture different aspects (good for clustering)\n")

plt.figure(figsize=(10, 8))
correlation = customer_df[rfm_features].corr()
sns.heatmap(correlation, 
            annot=True, 
            fmt='.2f',
            cmap='coolwarm',
            center=0,
            square=True,
            linewidths=1,
            cbar_kws={"shrink": 0.8})
plt.title('Feature Correlation Matrix', fontsize=14, pad=20)
plt.tight_layout()
plt.show()

# Interpret correlations
print("\n🔍 Correlation Insights:")
strong_corr = []
for i in range(len(correlation.columns)):
    for j in range(i+1, len(correlation.columns)):
        corr_val = correlation.iloc[i, j]
        if abs(corr_val) > 0.5:
            strong_corr.append((correlation.columns[i], correlation.columns[j], corr_val))

if strong_corr:
    print("  Strong correlations found:")
    for feat1, feat2, corr in strong_corr:
        print(f"    - {feat1} ↔ {feat2}: {corr:.2f}")
else:
    print("  ✓ No strong correlations - features capture different customer aspects")
    print("  ✓ Good for clustering - each feature adds unique information")

### 🎯 Outlier Analysis

**Important:** Outliers in customer data are often VALID extreme values, not errors.

Examples:
- A customer with 100 transactions is not an "outlier" - they're a loyal customer!
- A customer spending ₹1,000,000 is not an error - they're a high-value customer!

**Our Approach:**
1. Identify statistical outliers
2. Analyze if they represent valid customer segments
3. Keep them (clustering will naturally group them)

In [ ]:
# Outlier detection using IQR method
def detect_outliers_iqr(df, columns):
    """
    Detect outliers using Interquartile Range (IQR) method
    
    Outlier if: value < Q1 - 1.5*IQR  OR  value > Q3 + 1.5*IQR
    """
    outlier_report = []
    
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        n_outliers = len(outliers)
        pct_outliers = (n_outliers / len(df)) * 100
        
        outlier_report.append({
            'Feature': col,
            'Lower_Bound': round(lower_bound, 2),
            'Upper_Bound': round(upper_bound, 2),
            'N_Outliers': n_outliers,
            'Pct_Outliers': round(pct_outliers, 2)
        })
    
    return pd.DataFrame(outlier_report)

outlier_analysis = detect_outliers_iqr(customer_df, rfm_features)

print("🔍 Statistical Outlier Detection (IQR Method)")
print("=" * 80)
print(outlier_analysis)

print("\n💡 Interpretation:")
print("  These 'outliers' often represent important customer segments:")
print("    - High frequency: Loyal, engaged customers")
print("    - High monetary: High-value customers (VIP)")
print("    - High recency: Dormant/churned customers (re-activation opportunity)")
print("\n  Decision: Keep all data - clustering will naturally segment them")

In [ ]:
# Visualize distributions with outliers
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for idx, feature in enumerate(rfm_features):
    ax = axes[idx]
    
    # Boxplot
    bp = ax.boxplot(customer_df[feature], vert=True, patch_artist=True)
    bp['boxes'][0].set_facecolor('lightblue')
    bp['medians'][0].set_color('red')
    bp['medians'][0].set_linewidth(2)
    
    ax.set_ylabel(feature)
    ax.set_title(f'{feature} Distribution with Outliers')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Box Plot Interpretation:")
print("  - Box: 50% of customers (Q1 to Q3)")
print("  - Red line: Median")
print("  - Whiskers: 1.5 × IQR from box edges")
print("  - Dots beyond whiskers: Statistical 'outliers'")
print("\n  Note: Many 'outliers' visible - these are valid extreme customers!")

### ✅ Feature Engineering Complete!

**What we created:**
1. ✓ **Recency** - Days since last transaction (CORRECTLY calculated!)
2. ✓ **Frequency** - Number of transactions per customer
3. ✓ **Monetary** - Total and average spending
4. ✓ **Account Balance** - Current financial status
5. ✓ **Demographics** - Age, Gender, Location
6. ✓ **Additional metrics** - Transaction volatility (std)

**Key Insights So Far:**
- Dataset has typical long-tail distribution (few high-value customers)
- Features have low correlation (good for clustering)
- Many "statistical outliers" that are actually valuable customer segments

**Final Dataset:**

In [ ]:
# Display final customer dataset
print("📊 Final Customer-Level Dataset")
print("=" * 80)
print(f"Shape: {customer_df.shape[0]:,} customers × {customer_df.shape[1]} features")
print(f"\nFeatures:")
for col in customer_df.columns:
    print(f"  - {col}")

print("\n" + "=" * 80)
customer_df.head(10)

In [ ]:
# Save customer dataset for next part
customer_df.to_csv('customer_rfm_features.csv', index=False)
print("\n💾 Customer dataset saved to 'customer_rfm_features.csv'")
print("   Ready for exploratory analysis and clustering!")

---

## 🎯 Next Steps (Part 3):

1. Exploratory Data Analysis on customer features
2. Feature preparation for clustering
3. Optimal cluster selection
4. K-Means clustering
5. Business interpretation

---

### 📚 Key Takeaways from Part 2:

**Critical Fix Applied:**
- ✅ RFM Recency now correctly calculated as days since last transaction
- ❌ Original: Used time between first and last transaction (WRONG!)

**Best Practices Demonstrated:**
- Proper aggregation from transaction to customer level
- Thoughtful feature engineering with business context
- Statistical analysis before making decisions
- Comprehensive documentation of methodology

**Ready for Clustering:**
- Clean customer-level dataset
- Well-understood feature distributions
- Valid outliers preserved for segmentation